# Вебинар 1. Введение, примеры задач, бизнес- и ML-метрики 

# Домашнее задание 

**1) Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)**

**2) Доделать все функции, где стоит комментарий "сделать дома"**

### **1. Дополнительные метрики**

**Метрики на основе ранговой корреляции**
- Ранговый коэффициент корреляции Кендэлла
- Ранговый коэффициент корреляции Спирмена

**Метрики на основе каскадной модели поведения**
- Expected reciprocal rank (ERR@k)
- PFound

**Метрики новизны**
- Coverage — доля товаров, которая выдается рекомендателем
- Personalization — насколько различаются рекомендации между пользователями
- Diversity — насколько разнообразные товары находятся внутри рекомендации

---
Источники: 

https://habr.com/ru/company/econtenta/blog/303458/

https://temofeev.ru/info/articles/rekomendatelnye-sistemy-idei-podkhody-zadachi/

https://repositorio.uam.es/bitstream/handle/10486/665149/rank_vargas_recsys_2011_ps.pdf;jsessionid=6D86274434E5E27CEEA7F9670FD78EB4?sequence=1 

### 2. ML-mетрики качества

In [29]:
import pandas as pd
import numpy as np

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дорогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [30]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return (flags.sum() > 0) * 1

In [31]:
def hit_rate_at_k(recommended_list, bought_list, k=5):   
    
    #   сделать в домашней работе    
    
    return hit_rate(recommended_list[:k], bought_list)

In [32]:
recommended_list = [143, 156, 1134, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [33]:
hit_rate(recommended_list, bought_list)

1

In [34]:
hit_rate_at_k(recommended_list, bought_list, k=3)

1

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [0, 1, 0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [35]:
recommended_list = [2, 3, 8, 6, 9, 2, 5, 10] #id товаров
bought_list = [1, 5, 6]

In [36]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    return flags.sum() / len(recommended_list)

In [37]:
precision(recommended_list, bought_list)

0.25

In [38]:
def precision_at_k(recommended_list, bought_list, k=5):    
    return precision(recommended_list[:k], bought_list)

In [39]:
precision_at_k(recommended_list, bought_list, k=3)

0.0

In [40]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    
    bought_list = np.array(bought_list)
    recommend_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommend_list, bought_list)
    
    precision = np.dot(flags, prices_recommended).sum() / prices_recommended.sum()
    
    return precision

In [41]:
prices = [23, 2233, 534, 56, 1]
money_precision_at_k(recommended_list, bought_list, prices)

0.01966982788900597

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [65]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)
    

def recall_at_k(recommended_list, bought_list, k=5):
    
    #    сделать дома    
    
    return recall(recommended_list[:k], bought_list)


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):  
    
    # сделать дома
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommended_list, bought_list) 
    
    return flags.dot(prices_recommended).sum() / prices_bought.sum()
    

In [43]:
recall(recommended_list, bought_list)

0.6666666666666666

In [44]:
recall_at_k(recommended_list, bought_list, k=100)

0.6666666666666666

In [66]:
prices_recommended = [23, 2233, 534]
prices_bought = [50, 2233, 101, 60]
money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=3)

0.21849427168576105

---

# Метрики ранжирования

Metrics: https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)

Если важен порядок рекомендаций.

## AP@k
AP@k - average precision at k

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка рекомендаций

In [46]:
recommended_list = [22, 21, 4, 1, 234, 232432, 234234, 666] #id товаров
bought_list = [1, 221, 3, 4,]

In [47]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
            
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_/amount_relevant  
    

In [48]:
ap_k(recommended_list, bought_list, k=5)

0.41666666666666663

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [49]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                    ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 156, 991, 43, 11], # юзер 2
                       [1, 2]] # юзер 3

In [50]:
def map_k(recommended_list, bought_list, k=5):
    
    # сделать дома
    
    users_sum_ = sum([ap_k(recommended_list[user], bought_list[user], k=k) for user in range(len(bought_list))])
    result = users_sum_/ len(recommended_list)
    
    return result

In [51]:
map_k(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333

### AUC@k
AUC для первых k наблюдений  
- Можно посчитать как sklern.metrics.roc_auc_score для топ-k предсказаний
- Показывает долю верно отранжированных товаров

### Normalized discounted cumulative gain ( NDCG@k)


$$DCG = \frac{1}{|r|} \sum_u{\frac{[bought fact]}{discount(i)}}$$  

$discount(i) = i$ if $i <= 2$,   
$discount(i) = log_2(i+1)$ if $i > 2$


(!) Считаем для первых k рекоммендаций   
(!) - существуют вариации с другими $discount(i)$  
i - ранк рекомендованного товара  
|r| - кол-во рекомендованных товаров 

$$NDCG = \frac{DCG}{ideal DCG}$$


$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [52]:
# по желанию

def ndcg_at_k(recommended_list, bought_list, k=5):
    
    bought_item = np.array(bought_list)
    recommended_item = np.array(recommended_list)[:k]

    flags = np.isin(bought_list, recommended_list)
    flags_ideal = np.ones(len(bought_list))

    sum_ = 0
    sum_ideal = 0

    for i in range(len(bought_list)):        
        if i < 2:
            sum_ += flags[i] / (i+1)
            sum_ideal += flags_ideal[i] / (i+1)
        else:
            sum_ += flags[i] / np.log(i+1)
            sum_ideal += flags_ideal[i] / np.log(i+1)
    
    dcg = sum_ / len(recommended_item)
    dcg_ideal = sum_ideal / len(recommended_item)

    ndcg = dcg / dcg_ideal

    return ndcg

In [53]:
ndcg_at_k(recommended_list, bought_list, k=1)

0.5496726290767124

### Mean Reciprocal Rank ( MRR@k )


- Считаем для первых k рекоммендаций
- Найти ранк первого релевантного предсказания $k_u$
- Посчитать reciprocal rank = $\frac{1}{k_u}$

$$MRR = mean(\frac{1}{k_u})$$

In [54]:
def reciprocal_rank(recommended_list, bought_list, k=1):
    # сделать дома
    
    sum_ = 0
    
    for user in range(len(bought_list)):

        bought_item = np.array(bought_list[user])
        recommended_item = np.array(recommended_list[user])[:k]
         
        relevant_indexes = np.nonzero(np.isin(recommended_item, bought_item))[0] 
      
        if len(relevant_indexes) == 0:
            rr = 0
        else:
            first_relevant = relevant_indexes[0] + 1  
            rr = 1 / first_relevant

        sum_ += rr
    
    result = sum_/len(bought_list)

    return result

In [55]:
reciprocal_rank(recommended_list_3_users, bought_list_3_users, k=5)

0.3333333333333333